In [18]:
from keras.layers import Conv2D, Conv2DTranspose, Flatten, Input, Dense, Reshape
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.datasets import mnist
from keras import Model
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
import glob
import cv2
%matplotlib inline

# Train Data

In [2]:
%%time 
img_size = 64

samples = glob.glob('./lfwcrop_grey/faces/*')
data = []
print('Point 1')
for s in samples:
    img = mpimg.imread(s)
    data.append(np.expand_dims(img, 0))

Point 1
Wall time: 2min 13s


In [3]:
%%time 
data2 = []
for img in glob.glob("./FilmsFaceDatabase/s*/*.pgm"):
    img_read = mpimg.imread(img)
    img_read = cv2.resize(img_read, (64, 64))
    data2.append(np.expand_dims(img_read, 0))

Wall time: 4.12 s


In [4]:
%%time 
data3 = []
for img in glob.glob("./ufi-cropped/train/s*/*.pgm"):
    n = mpimg.imread(img)
    n = cv2.resize(n, (64, 64))
    data3.append(np.expand_dims(n,0))

Wall time: 21.2 s


In [25]:
%%time 
data4 = []
for img in glob.glob("./UTKFace/*"):
    n = mpimg.imread(img)
    
    n = cv2.cvtColor(n, cv2.COLOR_BGR2RGB) # конвертируем изображение в RGB
    n = cv2.cvtColor(n, cv2.COLOR_RGB2GRAY) # делаем изображение ЧБ
    
    n = cv2.resize(n, (64, 64))
    data4.append(np.expand_dims(n,0))

Wall time: 32.6 s


In [28]:
full_data = data+data2+data3+data4

In [29]:
faces = np.concatenate(full_data, axis=0)

In [30]:
faces.shape

(37094, 64, 64)

In [31]:
faces = np.expand_dims(faces, -1)

In [32]:
# prepare data
faces = faces / 255.

In [33]:
faces.shape

(37094, 64, 64, 1)

#### Fit autoencoder

In [34]:
# encoder
input_ = Input((64, 64, 1)) # 64
x = Conv2D(filters=8, kernel_size=2, strides=2, activation='relu')(input_) # 32
x = Conv2D(filters=16, kernel_size=2, strides=2, activation='relu')(x) # 16
x = Conv2D(filters=32, kernel_size=2, strides=2, activation='relu')(x) # 8
x = Conv2D(filters=64, kernel_size=2, strides=2, activation='relu')(x) # 4
x = Conv2D(filters=128, kernel_size=2, strides=2, activation='relu')(x) # 2
flat = Flatten()(x)
latent = Dense(128)(flat)

# decoder
reshape = Reshape((2,2,32)) #2
conv_2t_1 = Conv2DTranspose(filters=128, kernel_size=2, strides=2, activation='relu') # 4
conv_2t_2 = Conv2DTranspose(filters=64, kernel_size=2, strides=2, activation='relu') # 8
conv_2t_3 = Conv2DTranspose(filters=32, kernel_size=2, strides=2, activation='relu') # 16
conv_2t_4 = Conv2DTranspose(filters=16, kernel_size=2, strides=2, activation='relu') # 32
conv_2t_5 = Conv2DTranspose(filters=1, kernel_size=2, strides=2, activation='sigmoid') # 64

x = reshape(latent)
x = conv_2t_1(x)
x = conv_2t_2(x)
x = conv_2t_3(x)
x = conv_2t_4(x)
decoded = conv_2t_5(x) # 64

autoencoder = Model(input_, decoded)
encoder = Model(input_, latent)

decoder_input = Input((128,))
x_ = reshape(decoder_input)
x_ = conv_2t_1(x_)
x_ = conv_2t_2(x_)
x_ = conv_2t_3(x_)
x_ = conv_2t_4(x_)
decoded_ = conv_2t_5(x_) # 64
decoder = Model(decoder_input, decoded_)

In [35]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 64, 64, 1)         0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 32, 32, 8)         40        
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 16, 16, 16)        528       
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 8, 8, 32)          2080      
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 4, 4, 64)          8256      
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 2, 2, 128)         32896     
_________________________________________________________________
flatten_3 (Flatten)          (None, 512)               0         
__________

In [36]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [40]:
autoencoder.fit(faces, faces, epochs=10)

Epoch 1/10
37094/37094 [==============================] - 21s 573us/step - loss: 0.5981
Epoch 2/10
37094/37094 [==============================] - 23s 615us/step - loss: 0.5981
Epoch 3/10
37094/37094 [==============================] - 21s 575us/step - loss: 0.5981
Epoch 4/10
37094/37094 [==============================] - 23s 608us/step - loss: 0.5981
Epoch 5/10
37094/37094 [==============================] - 21s 571us/step - loss: 0.5981
Epoch 6/10
37094/37094 [==============================] - 21s 564us/step - loss: 0.5981
Epoch 7/10
37094/37094 [==============================] - 21s 565us/step - loss: 0.5981
Epoch 8/10
37094/37094 [==============================] - 21s 567us/step - loss: 0.5981
Epoch 9/10
37094/37094 [==============================] - 21s 563us/step - loss: 0.5981
Epoch 10/10
37094/37094 [==============================] - 21s 565us/step - loss: 0.5981


# Save model

In [41]:
# save weights
encoder.save_weights('encoder_weights_mri.h5')
decoder.save_weights('decoder_weights_mri.h5')

# save architecture
json_encoder = encoder.to_json()
json_decoder = decoder.to_json()

with open('encoder_mri_json.txt', 'w') as file:
    file.write(json_encoder)

with open('decoder_mri_json.txt', 'w') as file:
    file.write(json_decoder)